<a href="https://colab.research.google.com/github/pollinations/pollinations/blob/feature%2Fcolab-connection-loss-indication/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pollinator

Does the heavy-lifting for **Pollinations.AI** on Google Colab's free GPUs.

Click *Runtime - Run All*, switch back to the **Pollinations** tab of the browser and wait a little for the connection to be established.

![](https://pollinations.ai/Runtime-RunAll.png?)

In [1]:
#@title Config and Requirements

!pip install --upgrade pip
!pip install --upgrade typing-extensions==3.10.0.2
!pip install papermill json5
from IPython.display import clear_output
from os.path import join, basename
from shlex import quote

from glob import glob
from json5 import loads

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

#clear_output()
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
ipfs_root = "/content/ipfs"
worker_root = "/content"
notebook_path = join(ipfs_root,"input", "notebook.ipynb")

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}")

---Settings---
IPFS_ROOT: /content/ipfs


In [2]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output

clear_output()

In [3]:
#@title Install Pollinations CLI

%cd /content
!git clone https://github.com/pollinations/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout feature/colab-connection-loss-indication
!git pull
!npm run install_backend
%cd -

#clear_output()

/content
fatal: destination path 'pollinations' already exists and is not an empty directory.
/content/pollinations/app
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 14 (delta 7), reused 14 (delta 7), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/pollinations/pollinations
   06096cc..1ae89a5  feature/colab-connection-loss-indication -> origin/feature/colab-connection-loss-indication
Updating 06096cc..1ae89a5
Fast-forward
 app/src/network/useColab.js | 18 ++++++++++--------
 app/src/pages/Model.js      | 13 ++++++-------
 2 files changed, 16 insertions(+), 15 deletions(-)
Already on 'feature/colab-connection-loss-indication'
Your branch is up to date with 'origin/feature/colab-connection-loss-indication'.
Already up to date.

> pollinations@0.4.8 install_backend /content/pollinations/app
> cp -v dist/pollinate.js /usr/local/bin/pollinate && cp -v scripts/*.s

In [4]:
#@title Install and Run IPFS daemon &

#!/content/pollinations/ipfs/ipfsd_install_colab.sh
#!nohup /content/pollinations/ipfs/ipfsd_forever.sh &
#!sleep 5
clear_output()


node_id=!tr -dc A-Za-z0-9 </dev/urandom | head -c 13 ; echo ''

node_id=node_id[0]

gpu_description=!nvidia-smi -L
gpu_description=gpu_description[0]


print("---Node ID---:", node_id, gpu_description)

---Node ID---: eyYyMHO8wVne8 GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-44fbc6d6-7b66-8c25-fe20-c65351494b66)


In [5]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}&gpu={gpu_description}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}&gpu={gpu_description}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))




<IPython.core.display.Javascript object>

In [ ]:
#@title Notebook Runner
from time import sleep
import sys
import matplotlib
matplotlib.use('Agg')
output_path = ipfs_root+"/output"

!DEBUG=* pollinate --execute "run_notebook.sh" -l $output_path/log -p $ipfs_root --ipns -n $node_id --debounce 100 > /content/cid


  pollinate CLI options {
  path: '/content/ipfs',
  receive: false,
  send: false,
  once: false,
  ipns: true,
  nodeid: 'eyYyMHO8wVne8',
  debounce: '100',
  execute: 'run_notebook.sh',
  logout: '/content/ipfs/output/log'
} +0ms
  ipfsConnector localhost:5001 is not reachable. Connecting to https://ipfs.pollinations.ai +0ms
  ipfs:pubsub Creating publisher for eyYyMHO8wVne8 /output +0ms
  ipfs:pubsub Subscribing to pubsub events from eyYyMHO8wVne8 /input +0ms
  ipfs:pubsub Set heartbeat timeout. Waiting  30  seconds until next heartbeat +0ms
  ipfs:pubsub Executing subscribe eyYyMHO8wVne8/input +1ms
